<a href="https://colab.research.google.com/github/cytoscape/py4cytoscape/blob/0.0.10/doc/tutorials/Filtering-Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Filtering Networks
**Kozo Nishida, Kristina Hanspers, Alexander Pico, Barry Demchak**

**Package**

py4cytoscape 0.0.10

This vignette will introduce you to some techniques for filtering a network based on node properties. You will learn to:

- Select a set of nodes based on node degree and attribute filters
- Create a subnetwork based on selected nodes
- Hide a set of nodes based on filters

For this tutorial, we will use data from the STRING database (https://string-db.org/).

## Prerequisites

In addition to this package (py4cytoscape), you will need:

- Cytoscape 3.8 or greater, which can be downloaded from https://cytoscape.org/download.html. Simply follow the installation instructions on screen.
- Complete installation wizard
- Launch Cytoscape

**NOTE: To run this notebook, you must manually start Cytoscape first -- don't proceed until you have started Cytoscape.**

### Setup required only in a remote notebook environment

If you are using a remote notebook environment such as Google Colab, please execute the cell below. (If you’re running on your local notebook, you don’t need to do that.)


In [ ]:
_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@0.0.10'
import requests
exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)
IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

### Sanity test to verify Cytoscape connection

By now, the connection to Cytoscape should be up and available. To verify this, try a simple operation that doesn't alter the state of Cytoscape.


In [ ]:
p4c.cytoscape_ping()
p4c.cytoscape_version_info()

## Get network from STRING
We are going to query the STRING Disease database for the term “breast cancer”. By default, the app pulls the top 100 human proteins associated with the disease along with edges having an evidence strength of 0.4 or greater:

In [ ]:
p4c.install_app('STRINGapp')

In [ ]:
string_cmd = 'string disease query disease="breast cancer"'
p4c.commands_run(string_cmd)
string_net = p4c.get_network_suid()

In [ ]:
p4c.notebook_show_image()

## Filtering by degree
### Creating a degree filter
Every node in a network has a Degree property, which corresponds to the number of edges connecting the node to other nodes, either as a target or source. Filtering based on node degree is a useful way to remove nodes with too few (or too many) connections.

In this example we want to exclude low degree nodes, e.g., those with only 0, 1 or 2 connections:


In [ ]:
p4c.create_degree_filter('degree filter', [0,2], 'IS_NOT_BETWEEN')

In [ ]:
p4c.notebook_show_image()

At the bottom of the Select tab, you can see how many edges/nodes where selected.

### Creating a subnetwork from a selection

We can now create a new network, or subnetwork, from our selected set of nodes and all relevant edges:

In [ ]:
p4c.create_subnetwork(subnetwork_name='Breast cancer: highly connected nodes')

In [ ]:
p4c.notebook_show_image()

## Filtering by attribute

### Creating a column filter

We could also filter the network based on high disease score. The disease score comes from STRING and indicates the strength of the association to the disease queried.

Let’s select nodes from the original network with a disease score of greater than 4 (on a scale of 1-5):

In [ ]:
p4c.create_column_filter(filter_name='disease score filter', column='stringdb::disease score', criterion=4, predicate='GREATER_THAN', network=string_net)

In [ ]:
p4c.notebook_show_image()

Again, we can create a subnetwork from the selection:

In [ ]:
p4c.create_subnetwork(subnetwork_name='Breast cancer: high disease score')

In [ ]:
p4c.notebook_show_image()

## Combining filters
But what if we want to combine these two filters? You could apply them sequentially as individual filters, but then you’d need to be careful about the order in which you apply the filters. Alternatively, you can create a composite filter and apply the logic all at once!

Let’s combine the two filters “degree filter” and “disease score” to produce one filter, then apply it to the original network and create a final subnetwork:

In [ ]:
p4c.create_composite_filter('combined filter', ['degree filter','disease score filter'], network=string_net)

In [ ]:
p4c.create_subnetwork(subnetwork_name='final subnetwork')

In [ ]:
p4c.notebook_show_image()

We can apply a layout to help with interpreting the network:

In [ ]:
p4c.layout_network('force-directed defaultSpringCoefficient=5E-6')

In [ ]:
p4c.notebook_show_image()

This final network obviously contains fewer nodes than the original, but they are the most connected and most highly associated with the disease. If you examine the network you can see several well-known breast cancer oncogenes, for example BRCA1, TP53 and PTEN, near the center of the action.

## Hiding filtered nodes

As a final example of the filter functions, let’s return to the orignal network once more and apply our “combined filter”. But this time let’s *hide* the filtered out nodes, rather than forming a selection. This demonstrates the *applyFilter* function and the *hide* parameter that is optional for all *createXXXFilter* functions as well.

In [ ]:
p4c.apply_filter('combined filter', hide=True, network=string_net)

In [ ]:
p4c.notebook_show_image()